Python for Data Analysis
----

# Book
- CN: 利用Python进行数据分析 78.4MB.pdf
- EN: Python for Data Analysis 2nd Edition.pdf

![cover](images/cover1.png)


# 概述
原书英文版，2013年由OReilly出版，中文版由机械工业出版社出版。

全书12个章节：
- 准备工作
- 引言
- IPython
- Number基础
- pandas入门
- 数据加载、存储与文件格式
- 数据规整化
- 绘图和可视化
- 数据聚合与分组运算
- 时间序列
- 金融和经济数据
- NumPy高级应用
- 附录
 - Python语言精要


# 源代码

 `git clone https://github.com/pydata/pydata-book -b 1st-edition`

# 读书笔记
## Ch01 准备工作
本章节如题，即为学习如何来用Python来分析数据的准备工作。
- what： 处理对象是什么？ 主要是结构化数据表格
- How： 工具是什么？ Python， NumPy/Matplotlib/IPython/pandas/SciPy
- Why： Python简单易用，有强大的公共库资源
- Setup: 准备代码调试编写环境 

## Ch02 引言
### JSON 数据集

In [ ]:
#### Load JSON
import json
path = '/opt/Work/ML/pydata-book/ch02/usagov_bitly_data2012-03-16-1331923249.txt'
records = [json.loads(line) for line in open(path)]
records[0]

In [ ]:
type(records[0])

#time_zones = [rec['tz'] for rec in records]
time_zones = [rec['tz'] for rec in records if 'tz' in rec]
time_zones[:5]

In [ ]:
def get_counts(sequence):
    counts = {} # dict
    for x in sequence:
        if x in counts.keys():
            counts[x] += 1
        else:
            counts[x] = 1
    return counts
            
get_counts(time_zones)

In [ ]:
from collections import defaultdict

def get_counts2(sequence):
    counts = defaultdict(int)
    for x in sequence:
        counts[x] += 1
    return counts

get_counts2(time_zones)

In [ ]:
counts = get_counts2(time_zones)

In [ ]:
type(counts) #collections.defaultdict
type(counts.items()) # dict_items

In [ ]:
def top_counts(count_dict, n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]
counts = get_counts2(time_zones)
top_counts(counts)

In [ ]:
import pandas as pd; import numpy as np
from pandas import DataFrame, Series
frame = DataFrame(records)

In [ ]:
tz_counts = frame['tz'].value_counts()

In [ ]:
clean_tz = frame['tz'].fillna('Missing')

In [ ]:
type(clean_tz)

In [ ]:
# NB!!!
clean_tz[clean_tz==''] = 'Unknow'

In [ ]:
tz_counts = clean_tz.value_counts()
tz_counts[:10]

In [ ]:
frame['a'].head()
frame.a.head()

In [ ]:
results = Series(x.split()[0] for x in frame.a.dropna())
print(results.head(5))
print( results.value_counts()[:8] )

In [ ]:
# ?????
cframe = frame[frame.a.notnull()]
operating_system = np.where(cframe['a'].str.contains('Windows'),
                            'Windows', 'Not Windows')

In [ ]:
print( operating_system[:5])

In [ ]:
by_tz_os = cframe.groupby(['tz', operating_system])
agg_counts = by_tz_os.size().unstack().fillna(0)
agg_counts[:10]

In [ ]:
# 用于按照升序排列
indexer = agg_counts.sum(1).argsort()
indexer[:10]

In [ ]:
count_subset = agg_counts.take(indexer)[-10:]
count_subset

In [ ]:
count_subset.plot(kind='barh', stacked=True)

In [ ]:
normed_subset = count_subset.div(count_subset.sum(1), axis=0)
normed_subset.plot(kind='barh',stacked=True)

### MovieLens 1M数据集


In [ ]:
import pandas as pd
unames = ['user_id','gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames)

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames)

mnames = ['movie_id', 'title', 'genres']
movies=pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames)


In [ ]:
users.head()

In [ ]:
ratings.head()

In [ ]:
movies.head()

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)
data.head()

In [ ]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc=np.mean)
mean_ratings[:5]

In [ ]:
ratings_by_title = data.groupby('title').size()
ratings_by_title[:10]

In [ ]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

In [ ]:
mean_ratings = mean_ratings.ix[active_titles]
mean_ratings.head()

In [ ]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']
sorted_by_diff = mean_ratings.sort_index(by='diff')
sorted_by_diff.head()

In [ ]:
sorted_by_diff[::-1].head()

In [ ]:
# 根据电影名称分组的得分数据的标准差
rating_std_by_title = data.groupby('title')['rating'].std()

# 根据active_titles进行过滤
rating_std_by_title = rating_std_by_title.ix[active_titles]

# 根据值对series进行降序排列
rating_std_by_title.sort_values(ascending=False).head()

### 全美婴儿姓名分析
#### 1880-2010年间全美婴儿姓名

In [ ]:
! head -n 10 names/yob1881.txt

In [ ]:
import pandas as pd
names1880 = pd.read_csv('names/yob1880.txt', names=['name', 'sex', 'births'] )
names1880.head()

In [ ]:
names1880.groupby('sex')['births'].sum()

names1880.groupby('sex').births.sum()

In [ ]:
# 1880-2010
years = range(1880, 2011)
pieces = []
columns = ['name', 'sex', 'births']

for year in years:
    path = 'names/yob%d.txt' % year
    frame = pd.read_csv(path, names=columns)
    
    frame['year'] = year
    pieces.append(frame)
    
names = pd.concat(pieces, ignore_index=True)
names.head()

In [ ]:
names.groupby('sex')['births'].sum()

In [ ]:
total_births = names.pivot_table('births', index='year', columns='sex', aggfunc=sum)
total_births.tail()

In [ ]:
total_births.plot(title='Total births by sex and years')

In [ ]:
def add_prop(group):
    births = group.births.astype(float)
    group['prop'] = births / births.sum()
    return group
names = names.groupby(['year', 'sex']).apply(add_prop)
names.head()

In [ ]:
np.allclose(names.groupby(['year', 'sex']).prop.sum(), 1)

In [ ]:
g=names.groupby(['year', 'sex'])
type(g)

 
 ''' [:1000] cannot work as desired '''

def get_top1000(group):
    return group.sort_index(by='births', ascending=False)[:1000]

grouped = names.groupby(['year', 'sex'])
top1000 = grouped.apply(get_top1000)
top1000.head()

 			name 	sex 	births 	year 	prop
year 	sex 						
1947 	F 	431022 	Linda 	F 	99651 	1947 	0.056229
1948 	F 	441381 	Linda 	F 	96185 	1948 	0.056657
1947 	M 	437125 	James 	M 	94601 	1947 	0.051768
1957 	M 	544528 	Michael 	M 	92700 	1957 	0.043008
1947 	M 	437126 	Robert 	M 	91557 	1947 	0.050102

def get_top1000(group):
    #return group.sort_index(by='births', ascending=False)[:1000]
    return group

grouped = names.groupby(['year', 'sex'])
top1000 = grouped.apply(get_top1000)
top1000.head()
top1000 = top1000.sort_index(by='births', ascending=False)[:1000]
top1000.head()

In [ ]:
def get_top1000(group):
    return group.sort_index(by='births', ascending=False)[:1000]

grouped = names.groupby(['year', 'sex'],as_index=False)
top1000 = grouped.apply(get_top1000)
top1000.head()

#### 分析命名趋势

In [ ]:
boys = top1000[top1000.sex == 'M']
girls= top1000[top1000.sex == 'F']

In [ ]:
total_births = top1000.pivot_table('births', index='year', columns='name',aggfunc=sum)
#totle_births = names.pivot_table('births', index='year', columns='name',aggfunc=sum)
total_births.head()

In [ ]:
subset = total_births[['John', 'Harry', 'Mary', 'Marilyn']]
#subset = total_births[['John', 'Mary']]
subset.plot(subplots=True, figsize=(12,10), grid=False, title="Number of births per year")

#### 评估命名多样性的增长

In [ ]:
table        = top1000.pivot_table('prop', index='year', columns='sex',aggfunc=sum)
table.plot(title='Sum of the table1000.prop by year and sex', yticks=np.linspace(0, 1.2,13), xticks=range(1880,2020,10) )

In [ ]:
boys[boys.year==1947].head()


In [ ]:
df = boys[boys.year==1947]

prop_cumsum = df.sort_index(by='prop', ascending=False).prop.cumsum()
prop_cumsum.head(10)

In [ ]:
prop_cumsum.searchsorted(0.5)

In [ ]:
df = boys[boys.year == 1900]
in1900 = df.sort_index(by='prop', ascending=False).prop.cumsum()
in1900.searchsorted(0.5)+1

In [ ]:
def get_quantile_count(group, q=0.5):
    group = group.sort_index(by='prop', ascending=False)
    return group.prop.cumsum().searchsorted(q)+1
diversity = top1000.groupby(['year', 'sex']).apply(get_quantile_count)
diversity = diversity.unstack('sex')
diversity.head()

In [ ]:
diversity.plot(title='Number of polular names in top 50%')

#### 最后一个字母的变革

In [ ]:
get_last_letter = lambda x: x[-1]
last_letters = names.name.map(get_last_letter)
last_letters.name = 'last_letters'
table = names.pivot_table('births', index=last_letters, columns=['sex', 'year'], aggfunc=sum)

In [ ]:
# 我们选择代表性的三年
subtable = table.reindex(columns=[1910,1960,2010], level='year')
subtable.head()

In [ ]:
subtable.sum()

In [ ]:
letter_prop = subtable/subtable.sum().astype(float)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2,1, figsize=(10,8))
letter_prop['M'].plot(kind='bar', rot=0, ax=axes[0], title='Male')
letter_prop['F'].plot(kind='bar', rot=0, ax=axes[1], title='Female', legend=False)


In [ ]:
# 选几个典型的字母 d n y
letter_prop = table/table.sum().astype(float)
dny_ts = letter_prop.ix[['d','n','y'], 'M'].T
dny_ts.head()

In [ ]:
dny_ts.plot()

#### 男孩 ---> 女孩


In [ ]:
all_names = top1000.name.unique()
mask = np.array(['lesl' in x.lower() for x in all_names])

In [ ]:
all_names.shape

In [ ]:
lesley_like = all_names[mask]
lesley_like

In [ ]:
filtered = top1000[top1000.name.isin(lesley_like)]
filtered.groupby('name').births.sum()

In [ ]:
table = filtered.pivot_table('births', index='year', columns='sex', aggfunc=sum)
table = table.div(table.sum(1), axis=0)
table.tail()

In [ ]:
table.plot(style={'M': 'k-', 'F':'k--'})

#### 总结

此一章主要介绍了DF的用法，常规操作能解决很多问题
- 分组
- 统计
- 透视图
- 画图
    - pd.plot
    - matplotlib.pyplot.subplot
    
编写或者说抄写代码的时候才发现问题。比如，
+ 1. 区别：
    * groupby(['year', 'sex'])
    * groupby(['year', 'sex'], asindex=False)
+ 2. 取列：
    * names.year
    * names['year']
+ 3. 筛选
    * names[names.year==1880]
+ 4. 文件
    * [ json.loads(line) for l in open('some/file/path') ]

## IPython

## Number基础

#### ndarray: 一种多维数组对象
- create narray
    - array
    - asarray
    - arange
    - ones/ ones_like
    - zeros/ zeros_like
    - empty/ empty_like
    - eye/ identity
- methods
    - random
    - shape
    - reshape
    - ndim
    - dtype

In [ ]:
import numpy as np
data = np.random.rand(2,3)

In [ ]:
data
data*10
data.shape
data.dtype

In [ ]:
data1 = [6, 7.5, 8, 0, 1]
arr1 = np.array(data1)
arr1
arr1.shape

In [ ]:

data2 = [[1,2,3,4], [5,6,7,8]]
arr2 = np.array(data2) 
arr2.ndim  # 2
arr2.shape # (2,4)
arr2.dtype #int64

In [ ]:
np.zeros((3,6))

In [ ]:
np.empty((2,3,2))

In [ ]:
np.arange(15)

In [ ]:
arr1 = np.array([1,2,3], dtype=np.float64)
arr2 = np.array([1,2,3], dtype=np.int32)

In [ ]:
arr1.dtype

In [ ]:
arr2.dtype

In [ ]:
arr = np.array([1,2,3,4,5])
arr.dtype

In [ ]:
float_arr = arr.astype(np.float64)
float_arr.dtype

In [ ]:
numberic_strings = np.array(['1.25','-9.6', '42'], dtype=np.string_)
numberic_strings.astype(float)

#### 数组和标量之间的运算

In [ ]:
import numpy as np
arr = np.array([[1.,2,3],[4.,5.,6.]])
arr

In [ ]:
arr*arr

In [ ]:
1/arr

#### 基本的索引和切片

In [ ]:
arr = np.arange(10)
arr

In [ ]:
arr[5:8]

In [ ]:
arr[5:8] = 12
arr

In [ ]:
arr_slice = arr[5:8]
arr_slice[1] = 12345
arr

In [ ]:
arr_slice[:] = 666
arr

In [ ]:
arr2d = np.array([[1,2,3],[4,5,6],[7,8,9]])
arr2d

In [ ]:
arr2d[1][2]  ==  arr2d[1, 2]

In [ ]:
arr3d = np.array([ 
    [ [1,2,3],[4,5,6] ], 
    [ [7,8,9], [10,11,12] ]
])
arr3d

In [ ]:
arr3d.shape

In [ ]:
arr3d[0]

In [ ]:
arr3d[0] = 666
arr3d
arr3d[1,0]

##### 切片索引

In [ ]:
arr[1:6]
arr2d[:2]
arr2d[:2, 1:]
arr2d[:, :1]

#### 布尔型索引

In [ ]:
import math
names = np.array(['Bob', 'Joe', 'Will', 'Bob', 'Joe', 'Will','Joe'])
data = np.random.randn(7,4)

In [ ]:
names

In [ ]:
data

In [ ]:
names == 'Bob'

In [ ]:
mask = (names=='Bob') | (names=='Joe')
print(mask)
print(data.shape)
print(mask.shape)
data[mask]

In [ ]:
data[data < 0 ] = 0
data

#### 花式索引
花式索引和切片不一样，它总是将数据复制到新数组中。

In [ ]:
arr = np.empty((8,4))
for i in range(8):
    arr[i] = i
arr

In [ ]:
arr[[4, 3, 0, 6]]

In [ ]:
arr[[-3, -5, -7]]

In [ ]:
arr = np.arange(32).reshape((8,4))
arr

In [ ]:
arr[[1,5,7,2], [0,3,1,2]]

In [ ]:
arr[[1,5,7,2]] [: , [0,3,1,2]]

In [ ]:
arr[ np.ix_([1,5,7,2], [0,3,1,2]) ]

### 数组转置和轴对换
- transpose 转置
- 轴对换
- 内积 np.dot

In [ ]:
arr = np.arange(15).reshape((3,5))
arr

In [ ]:
arr.T

In [ ]:
arr = np.arange(16).reshape((2,2,4))
arr.transpose((1,0,2))

In [ ]:
arr = np.arange(16).reshape((2,2,4))
arr.swapaxes(1,2)

### 通用函数： 快速的元素级数组函数
- 一元ufunc
    - ads, fabs
    - sqrt
    - square
    - exp
    - log, log10, log2, log1p
    -sign
    - ceil
    - floor
    - rint
    - modf
    - isnan
    - isfinite, isinf
    - cos, cosh, sin, sinh,tan, tanh
    - arccos, arccosh, arcsin, arcsinh, arctan, arctanh
    - logical_not
    
- 二元ufunc
    - add
    - subtract
    - multiply
    - divide, floor_divide
    - power
    - maximum, fmax
    - minimum, fmin
    - mod
    - copysign
    - greater, greater_equal
    - less, less_equal
    - equal, not_equal
    - logical_and, logical_or, logical_xor
    

In [ ]:
arr = np.arange(10)
np.sqrt(arr)

In [ ]:
x = np.random.randn(8)
y = np.random.randn(8)
print(x)
print(y)
np.maximum(x,y)

In [ ]:
arr = np.random.randn(7)*5
np.modf(arr)

### 利用数组进行数据处理

In [ ]:
points = np.arange(-5, 5 , 0.01)
xs, ys = np.meshgrid(points, points)
z = np.sqrt(xs**2+ys**2)

import matplotlib.pyplot as plt
plt.imshow(z, cmap=plt.cm.gray);plt.colorbar()
plt.title('Image plot of $\sqrt{x^2+y^2}$ for a grid of values')


### 将条件逻辑表述为数组运算
布尔值在计算的过程中可以被当作0或者1来使用


In [ ]:
xarr = np.array([1.1,1.2,1.3,1.4,1.5])
yarr = np.array([2.1,2.2,2.3,2.4,2.5])
cond = np.array([True, False, True,True, False])
result = [
    (x if c else y)
          for x, y, c in zip(xarr, yarr, cond)
         ]
result

In [ ]:
arr = np.random.randn(4,4)
arr
np.where(arr>0, 2, -2) # 正数设置为2，负数设置为-2
np.where(arr>0, 2, arr)

In [ ]:
cond1 = np.array([True, False, False])
cond2 = np.array([False, True, False])
cond3 = np.array([False, False, True])

n =3
result = []
for i in range(n):
    if cond1[i] and cond2[i]:
        result.append(0)
    elif cond1[i]:
        result.append(1)
    elif cond2[i]:
        result.append(2)
    else:
        result.append(3)
print(result)

#等价于
np.where(cond1 & cond2, 0,
        np.where(cond1, 1, 
                 np.where(cond2, 2, 3)
        )
)

#等价于
#result = 1 * (cond1 -cond2) +2 * (cond2 & -cond1)+3 * -(cond1 | cond2)
for i in range(n):
    c1 = True if cond1[i]  else False
    c2 = True if cond2[i]  else False
    result = 1 * (c1 -c2) +2 * (c2 & -c1)+3 * -(c1 | c2)
    
        

In [ ]:
a = False
b = True
a-b
c = np.array([a,b])
np.logical_not(c)

### 数学和统计方法
- sum
- mean
- std, var
- min, max
- argmin, argmax
- cumsum, cumprod


In [ ]:
arr = np.random.randn(5,4)
arr.mean() == np.mean(arr)
arr.sum()

arr.mean(axis=1) == arr.mean(1)
arr.sum(axis=0) == arr.sum(0)

In [ ]:
arr = np.array([ [0,1,2], [3,4,5], [6,7,8] ])
arr.cumsum(0)

### 用于布尔型数组的方法

In [ ]:
arr = np.random.randn(100)
(arr>0).sum()

In [ ]:
bools = np.array( [False, False, True, False] )
bools.any()
bools.all()

### 排序

In [ ]:
arr  = np.random.randn(8)
arr.sort()
arr

In [ ]:
arr = np.random.randn(5,3)
arr.sort(1)
arr

In [ ]:
large_arr = np.random.randn(1000) 
large_arr.sort()
large_arr[int(0.05*len(large_arr))] #5%分位数

### 唯一化以及其他的集合逻辑
- unique(X)
- intersect1d(X,Y)
- union1d(X,Y)
- in1d(x, A)
- setdiff1d(X, Y)
- setxor1d(X, Y)

In [ ]:
names = np.array(['Bob', 'Joe', 'Will', 'Bob','Will', 'Joe'])
np.unique(names)
sorted(set(names))
(np.unique(names) == sorted(set(names))).all()



### 用于数组的文件输入输出
#### 将数组以二进制格式保存到磁盘

In [ ]:
arr = np.arange(10)
np.save('some_array', arr)
np.load('some_array.npy')

In [ ]:
#将多个数组保存到一个压缩文件中
np.savez('array_archive.npz', a=arr, b=arr)
arch = np.load('array_archive.npz')
arch['a']

#### 存取文本文件

In [ ]:
arr = np.loadtxt('array_ex.txt', delimiter=',')
pf = pd.read_csv('array_ex.txt')

### 线性代数

In [ ]:
x = np.array(
    [
        [1,2,3],
        [4,5,6]
    ],
    dtype=np.float
)

y = np.array(
    [
        [6,-23],
        [-1, 7],
        [8, 9]
    ],
    dtype = np.float
)

x.shape
y.shape
x.dot(y)
np.dot(x,y)

In [ ]:
np.dot(x, np.ones(3))

**numpy.linalg中有一组标准的矩阵分解运算以及诸如求逆和行列式之类的东西**

- diag
- dot
- trace
- det
- eig
- inv
- pinv
- qr
- svd
- solve
- lstsq

In [ ]:
from numpy.linalg import inv, qr
X = np.random.randn(5,5)

In [ ]:
mat = X.T.dot(X)
mat

In [ ]:
inv(mat)

In [ ]:
np.dot(mat, inv(mat))

In [ ]:
q,r = qr(mat)

In [ ]:
r

### 随机数生成

- seed
- permutation
- shuffle
- rand 均匀分布
- randint 整数
- randn （0，1）正态分布
- bioomial 二项式分布
- normal
- beta
- chisquare 卡方分布
- gamma
- uniform [0,1]均匀分布


In [ ]:
sample = np.random.normal(size=(4,4))

In [ ]:
sample

In [ ]:
from random import normalvariate
N = 1000000
%timeit samples = [normalvariate(0,1) for _ in range(N)]

In [ ]:
%timeit samples = np.random.normal(size=N)

### 随机漫步

In [ ]:
import random
position = 0
walk = [position]
steps = 1000
for i in range(steps):
    step = 1 if random.randint(0,1) else -1
    position += step
    walk.append(position)


In [ ]:
nsteps = 1000
draws = np.random.randint(0,2,size=nsteps)
steps = np.where(draws>0, 1, -1)
walk = steps.cumsum()
print (walk.min())
print (walk.max())
(np.abs(walk) >= 10).argmax()

#### 一次模拟多个随机漫步

In [ ]:
nwalks = 5000
nsteps = 1000
draws = np.random.normal(0,2, size=(nwalks, nsteps)) # 0 or 1
steps = np.where(draws>0, -1, 1)
walks = steps.cumsum(1)
walks
walks.max()
walks.min()
hits30 = (np.abs(walks) >= 30).any(1)
hits30
hits30.sum()
crossing_times = (np.abs(walks[hits30]) >= 30).argmax(1)
crossing_times.mean()

steps = np.random.normal(loc=0, scale=0.25, size=(nwalks, nsteps))

## pandas入门


## 数据加载、存储与文件格式

## 数据规整化

## 绘图和可视化

## 数据聚合与分组运算

## 时间序列

## 金融和经济数据

## NumPy高级应用